In [ ]:
# # md5 : 속도보다 보안 but 보안도 이제는 떨어진다지만
# 테스트해보니 속도, 결과 crc32랑 차이없네?!
# import hashlib
# def md5(file1):
#     md5h = hashlib.md5()
#     with open(file1, "rb") as f:
#         for chunk in iter(lambda: f.read(4096), b""):
#             md5h.update(chunk)
#     return md5h.hexdigest()

In [ ]:
import os
import pandas as pd
import file_function as ff
import re
import binascii # 내장모듈
import pickle # 내장모듈

### 중복파일 확인

In [ ]:
def crc32_checksum(filename):
    buf = open(filename,'rb').read()
    buf = (binascii.crc32(buf) & 0xFFFFFFFF)
    return "%08X" % buf

In [ ]:
def file_info(path:str) -> None:
    """
    하위경로포함 모든 파일에 대해 size,cre32로 중복 검사 후 
    모든 파일 정보 및 중복파일 목록을 excel, pickle로 바탕화면에 저장
    바로 중복파일 관리할 수 있도록 중복목록인 dict_sc 반환
    """

    dict_sc = {}  #중복파일끼리 묶을 딕셔너리
    df = None
    data = []
    
    df_matching = pd.read_excel(
        r'D:\workspace\python_work\파일\매각사 이름매칭.xlsx')

    for root, __dirs__, files in os.walk(path):
        
        for f in files:
            fullname = os.path.join(root, f)
            #key
            size = str(os.path.getsize(fullname))
            #value
            mtime = str(os.path.getmtime(fullname))
            crc32 = str(crc32_checksum(fullname))
            sell = ""
            for __index__, row in df_matching.iterrows():
                if re.search(str(row[0]), str(root)):  
                    sell = row[1]
                    break            
            filename = f
            ext = os.path.splitext(f)[1]
            length = len(filename)
            
            temp = {"size":size, "crc32": crc32, "mtime":mtime, "sell":sell, "root":root, "filename":filename, "ext":ext, "length":length, "fullname":fullname}
            data.append(temp)
            
            # 고유한 size, crc를 키로 하는 딕셔너리 만들기
            sc = size + crc32
            if sc not in dict_sc:
                dict_sc[sc] = [temp]
            else:
                dict_sc[sc].append(temp)
            
            
    df = pd.DataFrame(data)
    sc = df.duplicated(['size', 'crc32'], keep=False)
    sc.name = "sc"
    df = pd.concat([df, sc], axis=1)
    df_sc = df[df["sc"]]
    
    src_dir = r"C:\Users\SL\Desktop\중복 관리 파일"
    add_dir = path.split("\\")[-1]
    if not os.path.exists(os.path.join(src_dir,add_dir)):
        os.mkdir(os.path.join(src_dir,add_dir))
    
    # 파일 내보내기1 : path내 모든 파일
    df.to_excel(os.path.join(src_dir,add_dir,"파일정보.xlsx"))
    df.to_pickle(os.path.join(src_dir,add_dir,"파일정보.pkl"))
    
    # 파일 내보내기2 : 중복파일만 모은 거
    dict_sc2 = dict_sc.copy()
    for key, value in dict_sc.items():
        if len(value) == 1 :
            del dict_sc2[key]

    with open(os.path.join(src_dir,add_dir,"파일정보_sc.pkl"), "wb") as pkl :
        pickle.dump(dict_sc2, pkl)


In [ ]:
path = r'C:\Users\SL\Desktop\test'
file_info(path)

### 남길파일 이동파일 나누기

In [ ]:
with open(r"C:\Users\SL\Desktop\중복 관리 파일\test\파일정보_sc.pkl", "rb") as pkl :
    dict = pickle.load(pkl)

In [ ]:
# 채무자키 match
p_key = re.compile(r"[\d]{8}[\D]") 
# 사건번호
p_event = re.compile(r"[\D][\d]{4}[ㄱ-ㅎ가-힣]{1,3}[0-9]+")
# 문서구분
docu_kind = '원인서류|양도통지서|양통|승계|판결|명령|이행권고|화해권고|채권압류|압류|타채|결정|등본|초본|등,초본|등초본|외국인|개회|신복|파산'
p_docu = re.compile(docu_kind)
#-----------------------------------------------------------------------------#
# 사업자/주민번호
p_regit1 = re.compile(r'([\d]{3})-\d\d-\d\d\d\d\d[\D]+')
p_regit2 = re.compile(r'([\d]{6})-([\d]{6})[\D]+')
# 관리자키  match
p_admin10 = re.compile(r'[\d]{10}[\D]')
p_admin = re.compile(r'[\D][\d]{4}-[\d]{6}[\D]')

# 조건식 함수
def cond1(filename:str) : # 키 + [사건번호/문서구분]
    return (p_key.match(filename) != None) & ((p_event.search(filename) != None) | (p_docu.search(filename) != None))
def cond2(filename:str) : # [키]
    return (p_key.match(filename) != None) | (p_event.search(filename) != None)
def cond3(filename:str) : # [사업자/주민번호]
    return (p_regit1.search(filename) != None) | (p_regit2.search(filename) != None)
def cond4(filename:str) : # 관리자키
    return (p_admin10.match(filename) != None) | (p_admin.search(filename) != None)
def cond5(filename:str) : # 사건번호
    return (p_event.search(filename) != None)



In [ ]:
# 남길 파일 리스트
to_remain = []
# 이동관련
to_move = {}

for key, values in dict.items() :
    
    # length에 따라 역순정렬하기
    num = len(values)
    for i in range(num):
        indexOfGreater = i
        for j in range((i+1), num) :
            if values[indexOfGreater]["length"] < values[j]["length"] :
                values[i], values[j] = values[j], values[i]
    
    # stem과 fullname만 따로 임시 리스트에 저장
    filename_list = []
    fullname_list = []
    for v in values :
        filename_list.append(v["filename"]) 
        fullname_list.append(v["fullname"])
    
    # 유지할 것과 이동할 것 나누기
    isEnd = False
    for i in range(len(filename_list)) :
        if cond1(filename_list[i]) :
            to_remain.append(fullname_list[i])
            del fullname_list[i]
            to_move[filename_list[i]] = fullname_list
            isEnd = True
            break
        
    if not isEnd : #조건1을 만족하는 경우가 없었다면 다시 반복
        for i in range(len(filename_list)) :
            if cond2(filename_list[i]) :
                to_remain.append(fullname_list[i])
                del fullname_list[i]
                to_move[filename_list[i]] = fullname_list
                isEnd = True
                break
    
    if not isEnd :
        for i in range(len(filename_list)) :
            if cond3(filename_list[i]) :
                to_remain.append(fullname_list[i])
                del fullname_list[i]
                to_move[filename_list[i]] = fullname_list
                isEnd = True
                break
    if not isEnd :
        for i in range(len(filename_list)) :
            if cond4(filename_list[i]) :
                to_remain.append(fullname_list[i])
                del fullname_list[i]
                to_move[filename_list[i]] = fullname_list
                isEnd = True
                break
    if not isEnd :
        for i in range(len(filename_list)) :
            if cond5(filename_list[i]) :
                to_remain.append(fullname_list[i])
                del fullname_list[i]
                to_move[filename_list[i]] = fullname_list
                isEnd = True
                break   
    if not isEnd :
        to_remain.append(fullname_list[0])
        del fullname_list[0]
        to_move[filename_list[0]] = fullname_list


### 이동할 파일 옮기기

In [ ]:
dst_root = r'C:\Users\SL\Desktop\중복 관리 파일\test중복파일'

In [ ]:
for dst_dir, srcs in to_move.items() :
    dir = os.path.join(dst_root, dst_dir)
    if not os.path.exists(dir) :
        os.makedirs(dir)
    
    for src in srcs :
        #중복파일/남길파일명/파일명
        #원래폴더트리는 남길파일을 참고하거나, 파일로도 있음.(엑셀파일 올려두자)
        dst = os.path.join(dir, os.path.split(src)[1])
        os.rename(src, dst)
        
        